In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import cloudscraper

I use Beautiful Soup to do this challenge. The website I choose is Reformation. I scrapped 30 items in this notebook. This logic is as follows:
<br>Step 0: create empty list for all the information we want
<br>Step 1: scrape the content on the clothing page. Count the number of items we have
<br>Step 2: Loop through each item. For each item, I scrapped its individual link, and then find its detailed description. In each loop, append each piece of information we want to the empty list that I created in step 0
<br>Step 3: Using a pandas dataframe to store all lists. 


If we want to scrape all the items on the website, we can use nested loop.
<br>Step 0: scrape all element in the navigation bar on the top left cornor.
<br>Step 1: repeat all steps mentioned above in each navigation bar



After we finish all scrapping, we use psycopg2 to establish the connection between python and postgreSQL. We transform pandas dataframe to sql dataframe, and upload/update all information we have. 

In [2]:
scraper = cloudscraper.create_scraper()
url = "https://www.thereformation.com/clothing"
response = scraper.get(url)
if not response.status_code == 200:
    None
try:
    results_page = BeautifulSoup(response.content,'lxml') 
except:
    None


In [3]:
import numpy as np
from datetime import date

display_name_list = list()
product_material_list = list()
color_list = list()
size_list = list()
price_list = list()
product_url_list = list()
image_links_list = list() 
description_list = list()
scrapped_date_list = list()
gender_list = list()
secondhand_list = list() 

try:
    items = results_page.find_all('div', re.compile('product-grid__item'))
    for item in items:
        try:
            item_url = "https://www.thereformation.com/clothing" + item.find('a').get('href')
            product_url_list.append(item_url)
            item_response = requests.get(item_url)
            item_page = BeautifulSoup(item_response.content,'lxml') 
        except:
            break
        
        try:
            images = item.find('div',{'class':re.compile('product-tile__media-container')}).find_all('img')
            img_list = list()
            for image in images:
                img_list.append(image['data-src'])
            image_links_list.append(img_list)
        except:
            image_links_list.append(np.NaN)
            
        try:
            display_name = item_page.find('div',{'class': 'pdp__title'}).find('h1').get_text()[1:-1]
            display_name_list.append(display_name)
        except:
            display_name.append(np.NaN)
            
        try:
            fabric = item_page.find_all('div',{'class': re.compile('pdp__accordion-item.*')})[3]
            material = fabric.find('div',{'class': re.compile('margin-b.*')}).get_text()[1:-1]
            product_material_list.append(material)
        except:
            product_material_list.append(np.NaN)
        
        try:
            colors = list()
            color_buttons = item_page.find('div',{'class': re.compile('product-attribute product-attribute--color.*')}).find_all('button')
            for button in color_buttons:
                color = button.get_text().strip()
                colors.append(color)
            color_list.append(colors)
        except:
            color_list.append(np.NaN)
            
        try:
            sizes = list()
            size_buttons = item_page.find('div',{'class': re.compile('product-attribute product-attribute--size.*')}).find_all('button')
            for button in size_buttons:
                size = button.get_text()
                sizes.append(size)
            size_list.append(sizes)
        except:
            size_list.append(np.NaN)
            
        try:
            price = item_page.find('div',{'class': re.compile('pdp__price.*')}).find('span',{'class':'price--formated'}).get_text()[1:-1]
            price_list.append(price)
        except:
            price_list.append(np.NaN)
            
        try:
            description = item_page.find('div',{'class': re.compile('pdp__description.*')}).get_text().strip()
            description_list.append(description)
        except:
            description_list.append(np.NaN)
            
        scrapped_date_list.append(date.today())
        
        gender_list.append('women')
        
        secondhand_list.append(False)
        
except:
    None

In [4]:
df = pd.DataFrame({'product_url':product_url_list,
                   'item_name':display_name_list, 
                   'material':product_material_list, 
                   'colors':color_list, 
                   'sizes':size_list, 
                   'price':price_list,
                   'image_link':image_links_list,
                   'description':description_list,
                   'scrapped_date': scrapped_date_list,
                   'gender': gender_list,
                   'secondhand': secondhand_list})
df

,product_url,item_name,material,colors,sizes,price,image_link,description,scrapped_date,gender,secondhand
0,https://www.thereformation.com/clothing/produc...,Gemini Dress,Dry clean only,[Color:Green Check],"[XS, S, M, L, XL]",$218.00,[https://media.thereformation.com/image/list/f...,Short and sweet. The Gemini is a fully smocked...,2022-06-03,women,False
1,https://www.thereformation.com/clothing/produc...,Glory Dress,Dry clean only,[Color:Sea Green],"[XS, S, M, L, XL]",$278.00,[https://media.thereformation.com/image/list/f...,Have you romanced yourself lately? The Glory i...,2022-06-03,women,False
2,https://www.thereformation.com/clothing/produc...,Cowboy Striped High Rise Straight Jeans,Organic cotton doesn’t allow genetically modif...,[Color:Pinyon Stripe],"[23, 24, 25, 26, 27, 28, 29, 30, 31]",$188.00,[https://media.thereformation.com/image/list/f...,"Happy, sustainable butts. The Cowboy is slim f...",2022-06-03,women,False
3,https://www.thereformation.com/clothing/produc...,Lavinia Knit Top,Organic cotton doesn’t allow genetically modif...,"[Color:Black, Color:Ivory]","[XS, S, M, L, XL]",$68.00,[https://media.thereformation.com/image/list/f...,Throw on and go. The Lavinia is a fitted top w...,2022-06-03,women,False
4,https://www.thereformation.com/clothing/produc...,Tyra Knit Dress,Organic cotton doesn’t allow genetically modif...,"[Color:Black, Color:Corvette, Color:Ivory]","[XS, S, M, L, XL]",$148.00,[https://media.thereformation.com/image/list/f...,"Look cute, be comfortable. The Tyra is a sleev...",2022-06-03,women,False
5,https://www.thereformation.com/clothing/produc...,Mason Pant,"TENCEL™ Lyocell comes from Eucalyptus trees, w...","[Color:Army, Color:Black, Color:Cafe, Color:Kh...","[0, 2, 4, 6, 8, 10, 12]",$178.00,[https://media.thereformation.com/image/list/f...,Power pants. The Mason is slim fitting in the ...,2022-06-03,women,False
6,https://www.thereformation.com/clothing/produc...,Jeune Dress,Organic cotton doesn’t allow genetically modif...,"[Color:Black, Color:Layla, Color:Zest]","[0, 2, 4, 6, 8, 10, 12]",$218.00,[https://media.thereformation.com/image/list/f...,Short and sweet. The Jeune is a mini dress wit...,2022-06-03,women,False
7,https://www.thereformation.com/clothing/produc...,Rishi Dress,This is a lightweight voile fabric made with 5...,[Color:Ivory],"[0, 2, 4, 6, 8, 10, 12]",$278.00,[https://media.thereformation.com/image/list/f...,Detail oriented. The Rishi is a midi dress wit...,2022-06-03,women,False
8,https://www.thereformation.com/clothing/produc...,Cynthia High Rise Straight Cropped Jeans,Organic cotton doesn’t allow genetically modif...,"[Color:Colorado, Color:Tahoe]","[23, 24, 25, 26, 27, 28, 29, 30, 31]",$128.00,[https://media.thereformation.com/image/list/f...,"Smart ass jeans. This is a high rise, rigid je...",2022-06-03,women,False
9,https://www.thereformation.com/clothing/produc...,Juliette Dress,Viscose – aka rayon – is a man-made cellulosic...,"[Color:Black, Color:Emerald, Color:Formosa, Co...","[0, 2, 4, 6, 8, 10, 12]",$248.00,[https://media.thereformation.com/image/list/f...,Somebody's fancy. The Juliette is slim fitting...,2022-06-03,women,False


In [5]:
#!pip install psycopg2-binary
import psycopg2 as ps

In [6]:
keepalive_kwargs = {
  "keepalives": 1,
  "keepalives_idle": 60,
  "keepalives_interval": 10,
  "keepalives_count": 5
}

def connect_to_db(host_name,dbname, port, username, password):
    try:
        conn = ps.connect(host = host_name, database = dbname, user = username, password = password, port = port, **keepalive_kwargs)
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn

In [7]:
host_name = 'database2.cs7tcf7vaona.us-east-1.rds.amazonaws.com' #endpoint
dbname = 'database2'
port = '5432'
username = 'postgres'
password = 'A1205yiyao'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [8]:
def create_table(curr):
    create_table_command = """CREATE TABLE IF NOT EXISTS items1 (
                    product_url TEXT PRIMARY KEY,
                    item_name TEXT NOT NULL,
                    material TEXT NOT NULL,
                    colors TEXT NOT NULL,
                    sizes TEXT NOT NULL,
                    price TEXT NOT NULL,
                    image_link TEXT NOT NULL,
                    description TEXT NOT NULL,
                    scrapped_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    gender TEXT NOT NULL,
                    secondhand TEXT NOT NULL
            )"""
    curr.execute(create_table_command)

curr = conn.cursor()
create_table(curr)
conn.commit()

In [9]:
def check_if_item_exists(curr, product_url): 
    query = """SELECT product_url FROM items1 WHERE product_url = %s"""

    curr.execute(query, (product_url,))
    return curr.fetchone() is not None
curr = conn.cursor()
# check_if_item_exists(curr, 'haha')

In [10]:
def insert_into_table(curr, product_url, item_name, material, colors, sizes, price, image_link, description, scrapped_date, gender, secondhand):
    insert_into_items = """INSERT INTO items1 (product_url, item_name, material, colors, sizes, price,
                        image_link, description, scrapped_date, gender, secondhand)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
    row_to_insert = (product_url, item_name, material, colors, sizes, price, image_link, description, scrapped_date, gender, secondhand)
    curr.execute(insert_into_items, row_to_insert)

# insert_into_table(curr, 'product_url', 'item_name', 'material', ['black','white'], 'sizes', 'price', ['www','web'], 'description', date.today(), 'women', 'secondhand')
# conn.commit()

In [11]:
def update_row(curr, product_url, item_name, material, colors, sizes, price, image_link, description, scrapped_date, gender, secondhand):
    query = """UPDATE items1
            SET item_name = %s,
                material = %s,
                colors = %s,
                sizes = %s,
                price = %s,
                image_link = %s,
                description = %s,
                scrapped_date = %s,
                gender = %s,
                secondhand = %s
            WHERE product_url = %s;"""
    vars_to_update = (item_name, material, colors, sizes, price, image_link, description, scrapped_date, gender, secondhand, product_url)
    curr.execute(query, vars_to_update)

update_row(curr, 'product_url', 'item_name_1', 'new_material', ['black','white'], 'sizes', 'price', ['www','web'], 'description', date.today(), 'women', 'secondhand')
conn.commit()    

In [12]:
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['product_url','item_name', 'material', 'colors', 'sizes', 'price', 'image_link',
                                   'description', 'scrapped_date', 'gender', 'secondhand'])
    for i, row in df.iterrows():
        if check_if_item_exists(curr, row['product_url']): # If item already exists then we will update
            update_row(curr, row['product_url'], row['item_name'], row['material'], row['colors'], row['sizes'], row['price']
                          , row['image_link'], row['description'], row['scrapped_date'], row['gender'], row['secondhand'])
        else: # The item doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [13]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['product_url'], row['item_name'], row['material'], row['colors'], row['sizes'], row['price']
                          , row['image_link'], row['description'], row['scrapped_date'], row['gender'], row['secondhand'])

In [14]:
new_item_df = update_db(curr,df)
conn.commit()

In [15]:
append_from_df_to_db(curr, new_item_df)
conn.commit()

In [16]:
new_item_df = update_db(curr,df)
conn.commit()

In [17]:
curr.execute("SELECT * FROM items1")
print(curr.fetchall())

[('https://www.thereformation.com/clothing/products/gemini-dress/1310743GCK.html?dwvar_1310743GCK_color=GCK', 'Gemini Dress', 'Dry clean only', '{"Color:Green Check"}', '{XS,S,M,L,XL}', '$218.00', '{"https://media.thereformation.com/image/list/fn_select:jq:first(.%5B%5D%7Cif%20has(%22metadata%22)%20then%20select(any(.metadata%5B%5D;%20.external_id%20==%20%22sfcc-is-main%22%20and%20.value.value%20==%20%22True%22))%20else%20empty%20end)/f_auto,q_auto,dpr_auto/w_900/1310743-GCK.json?_s=RAABAB0","https://media.thereformation.com/image/list/fn_select:jq:first(.%5B%5D%7Cif%20has(%22metadata%22)%20then%20select(any(.metadata%5B%5D;%20.external_id%20==%20%22sfcc-gallery-position%22%20and%20.value%20==%202))%20else%20empty%20end)/f_auto,q_auto,dpr_auto/w_900/1310743-GCK.json?_s=RAABAB0"}', "Short and sweet. The Gemini is a fully smocked, mini dress with a straight neckline. It features adjustable spaghetti straps and a ruffled hem. It's fitted throughout giving you a little shape in the areas w